### ITCR - Escuela de Computación
### Curso IC-6200 - Inteligencia Artificial
### Aprendizaje supervisado

### Redes de memoria de corto y largo plazo con PyTorch 
### (Long-Short Term Memory Networks-LSTM)

**Profesora: María Auxiliadora Mora**

### Tarea corta #7

### Estudiantes:
**1. Pablo Alberto Muñoz Hidalgo**

**2. Luis Andrés Rojas Murillo**


## Introducción

La clasificación de textos y el reconocimiento de entidades nombradas (Named Entity Recognition o NER por sus siglas en inglés) son técnicas fundamentales que constituyen el primer paso en muchas tareas de Procesamiento de lenguaje natural (NLP). NER, es un área de investigación relacionada a la extracción de información, que permite localizar y clasificar nombres de entidades que se encuentran en texto libre, en categorías comunmente organizaciones, lugares, tiempo, personas, entre otros. Ejemplo:

- El fundador de [Microsoft Corporation] (organización), [Bill Gates] (persona), comentó que se abrirán 1000 puestos de trabajo en la [Región Chorotega] (lugar) a partir del año 2022 (fecha).  

La clasificación de textos permite categorizar el contenido asociando este a un conjunto de etiquetas predefinidas o clases. Su uso más popular es el análisis de sentimientos. Ejemplo:

- En mi opinión, la película fue muy buena porque pudo dar a conocer a los espectadores cómo puede afectar una situación traumática a la mente humana. (Clase = 5 o excelente). 

Las redes neuronales recurrentes o RNN (Rumelhart et al., 1986, como se citó en LeCun et al., 2015) son una familia de redes neuronales para el procesamiento de secuencias de datos, las cuales en un tiempo t, reciben el estado anterior, es decir, su salida en el tiempo t podría usarse como insumo del procesamiento de la siguiente entrada, de modo que la información pueda propagarse a medida que la red pasa por la secuencia de entrada. Las redes Long Short-Term Memory (LSTM) son un tipo de red neuronal recurrente capaz de aprender dependencias a largo plazo.

El siguiente ejemplo implementa NER con una LSTM para etiquetar el rol que juegan las palabras en las oraciones. 


## Ejemplo

El sistema implementado en el código adjunto soluciona el problema de estimar el rol de una palabra en una frase, por ejempo roles como determinante (DET), nombre (NN) y verbo (V). 
Ejemplo para la frase:

- "El perro come manzana" la salida deberá ser: ["DET", "NN", "V", "NN"]). 

Este proceso se conoce en el procesamiento de lenguaje natural como "part of speech tagging (POS)".

Este es un ejemplo simple con datos introducidos en el código basado en [1].

Se realizarán los siguientes pasos

   * Definición de los ejemplos (codificados) 
   * Preprocesamiento de las palabras a clasificar
   * Definición del modelo
   * Instanciación del modelo, definición de la función de pérdida y del optimizador  
   * Entrenamiento de la red
   * Pruebas del modelo resultante con unos cuantos ejemplos.


In [1]:
# Bibliotecas requeridas

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import csv
import random

torch.manual_seed(1)

In [2]:
# Funciones utilitarias

def max_values(x):
    """
    Retorna el valor máximo y en índice o la posición del valor en un vector x.
    Parámetros: 
        x: vector con los datos. 
    Salida: 
        out: valor 
        inds: índice
    """
    out, inds = torch.max(x,dim=1)   
    return out, inds
    

In [6]:
# Read the data from the CSV file
with open('data.csv', 'r') as file:
    reader = csv.reader(file)
    data = list(reader)

# Divide the data into training and testing sets
random.shuffle(data)
split = int(0.8 * len(data))
training_data = data[:split]
test_data = data[split:]

# Split the training data into text and labels
for i in range(len(training_data)):
    splitted_text = training_data[i][0].split()
    #put the rest of i in other list
    labels = []
    for j in range(len(training_data[i])):
        if j != 0:
            labels.append(training_data[i][j])
    training_data[i] = [0,0]
    training_data[i][0] = splitted_text
    training_data[i][1] = labels

# Split the test data into text and labels
for i in range(len(test_data)):
    splitted_text = test_data[i][0].split()
    #put the rest of i in other list
    labels = []
    for j in range(len(test_data[i])):
        if j != 0:
            labels.append(test_data[i][j])
    test_data[i] = [0,0]
    test_data[i][0] = splitted_text
    test_data[i][1] = labels

# print the data
print("Training data:")
print(training_data)
print("Test data:")
print(test_data)





Training data:
[[['los', 'estudiantes', 'estudian', 'en', 'la', 'biblioteca', 'de', 'la', 'universidad'], ['DET', 'NN', 'V', 'PREP', 'DET', 'NN', 'PREP', 'DET', 'NN', 'PREP', 'DET', 'NN']], [['la', 'casa', 'estÃ¡', 'en', 'la', 'colina'], ['DET', 'NN', 'V', 'PREP', 'DET', 'NN']], [['juan', 'y', 'marÃ\xada', 'bailan', 'en', 'la', 'fiesta'], ['NN', 'CONJ', 'NN', 'V', 'PREP', 'DET', 'NN']], [['ana', 'toca', 'el', 'piano', 'en', 'la', 'sala'], ['NN', 'V', 'DET', 'NN', 'PREP', 'DET', 'NN']], [['el', 'niÃ±o', 'juega', 'con', 'su', 'pelota', 'en', 'el', 'parque'], ['DET', 'NN', 'V', 'PREP', 'DET', 'NN', 'PRE']], [['marÃ\xada', 'y', 'pedro', 'toman', 'cafÃ©', 'en', 'el', 'cafÃ©', 'de', 'la', 'esquina'], ['NN', 'CONJ', 'NN', 'V', 'DET', 'NN', 'PREP', 'DET', 'NN', 'PREP', 'DET', 'NN']], [['marcos', 'lee', 'un', 'periÃ³dico', 'en', 'la', 'cafeterÃ\xada'], ['NN', 'V', 'DET', 'NN', 'PREP', 'DET', 'NN']], [['pablo', 'canta', 'en', 'la', 'ducha'], ['NN', 'V', 'PREP', 'DET', 'NN']], [['el', 'sol', 'bri

In [4]:
# Preparación de los datos 
def prepare_sequence(seq, to_ix):
    """
    Retorna un tensor con los indices del diccionario para cada palabras en una oración.
    Parámetros:
       seq: oración
       to_ix: diccionario de palabras.
    """
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)





# Frases de entrenamiento 
# El modelo solo va a aprender a identificar DET, NN, V 
training_data = [
    ("el perro come manzana".split(), ["DET", "NN", "V", "NN"]),
    ("Todos leen ese libro".split(), ["NN", "V", "DET", "NN"]),
    ("el gato corre".split(), ["DET", "NN", "V"]),
    ("el joven juega".split(), ["DET", "NN", "V"]),
    ("el gato brinca".split(), ["DET", "NN", "V"]),
    ("maria come manzana".split(), ["NN", "V", "NN"]),
    ("gloria hace la tarea".split(), ["NN", "V", "DET", "NN"]),
    ("ricardo hace la sopa".split(), ["NN", "V", "DET", "NN"]),
    ("marta toma cafe".split(), ["NN", "V", "NN"]),
    ("gloria hace la suma".split(), ["NN", "V", "DET", "NN"]),
    ("ricardo estudia espaniol".split(), ["NN", "V", "NN"])
]

# Datos de prueba
test_data = [
    ("el perro juega".split(), ["DET", "NN", "V"]),
    ("marta duerme la siesta".split(), ["NN", "V", "DET", "NN"]),
    ("maria come pastel".split(), ["NN", "V", "NN"]),
    ("el estudiante realiza el examen".split(), ["DET", "NN", "V", "DET", "NN"])
   ]

# Diccionario las palabras
word_to_ix = {}
for sent, tags in training_data + test_data:
    for word in sent:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)
            
print("Diccionario", word_to_ix)

# Asignar índices a las etiquetas
tag_to_ix = {"DET": 0, "NN": 1, "V": 2}



Diccionario {}


In [5]:
# Ejemplo de procesamiento de una oración
inputs = prepare_sequence(training_data[0][0], word_to_ix)
print(training_data[0][0])                          
print(inputs)

IndexError: list index out of range

In [ ]:
# Definición del modelo

# El modelo es una clase que debe heredar de nn.Module
class LSTMTagger(nn.Module):
    """
    Clase para aplicar POST a oraciones en español. 
    """
    
    # Incialización del modelo
    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        """
        Inicialización de la clase.
        Parámetros:
           embedding_dim: dimesionalidad del vector de palabras. 
           hidden_dim: dimensión de la capa oculta de la red. 
           vocab_size: tamaño del vocabulario.  
           tagset_size: número de clases.
        """
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim

        # Primero se pasa la entrada a través de una capa Embedding. 
        # Esta capa construye una representación de los tokens de 
        # un texto donde las palabras que tienen el mismo significado 
        # tienen una representación similar.
        
        # Esta capa captura mejor el contexto y son espacialmente 
        # más eficientes que las representaciones vectoriales (one-hot vector).
        # En Pytorch, se usa el módulo nn.Embedding para crear esta capa, 
        # que toma el tamaño del vocabulario y la longitud deseada del vector 
        # de palabras como entrada. Ejemplos en [3] y [4]
        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)

        # El LSTM toma word_embeddings como entrada y genera los estados ocultos
        # con dimensionalidad hidden_dim.  
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)

        # La capa lineal mapea el espacio de estado oculto 
        # al espacio de clases
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)

    def forward(self, sentence):
        # Pasada hacia adelante de la red. 
        # Parámetros:
        #    sentence: la oración a procesar
        embeds = self.word_embeddings(sentence)
        lstm_out, _ = self.lstm(embeds.view(len(sentence), 1, -1))
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))

        # Se utiliza softmax para devolver la probabilidad de cada etiqueta
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores


In [ ]:
# Instanciación del modelo, definición de la función de pérdida y del optimizador   

# Hiperparámetros de la red
# Valores generalmente altos (32 o 64 dimensiones).
EMBEDDING_DIM = 6
HIDDEN_DIM = 6

# Instancia del modelo
model = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM, len(word_to_ix), len(tag_to_ix))

# Función de pérdida: Negative Log Likelihood Loss (NLLL). 
# Generalmente utilizada en problemas de clasificacion con múltiples clases.
loss_function = nn.NLLLoss()

# Optimizador Stochastic Gradient Descent  
optimizer = optim.SGD(model.parameters(), lr=0.1)


In [ ]:
# Entrenamiento del modelo 

# Valores antes de entrenar
with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    tag_scores = model(inputs)
    
    print(training_data[0][0])
    
    # Clasificación    
    print(tag_scores)

# Épocas de entrenamiento
for epoch in range(200):  
    for sentence, tags in training_data:
        ## Paso 1. Pytorch acumula los gradientes.
        # Es necesario limpiarlos
        model.zero_grad()

        # Paso 2. Se preparan las entradas, es decir, se convierten a
        # tensores de índices de palabras.
        sentence_in = prepare_sequence(sentence, word_to_ix)
        targets = prepare_sequence(tags, tag_to_ix)

        # Paso 3. Se genera la predicción (forward pass).
        tag_scores = model(sentence_in)

        # Paso 4. se calcula la pérdida, los gradientes, y se actualizan los 
        # parámetros por medio del optimizador.
        loss = loss_function(tag_scores, targets)
        loss.backward()
        optimizer.step()

# Despligue de la puntuación luego del entrenamiento
with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    tag_scores = model(inputs)
   
    print("Resultados luego del entrenamiento para la primera frase")
    # Las palabras en una oración se pueden etiquetar de tres formas.
    # La primera oración tiene 4 palabras "El perro come manzana"
    # por eso el tensor de salida tiene 4 elementos. 
    # Cada elemento es un vector de pesos que indica cuál etiqueta tiene más
    # posibilidad de estar asociada a la palabra. Es decir hay que calcular 
    # la posición del valor máximo
    print(tag_scores)

['el', 'perro', 'come', 'manzana']
tensor([[-1.4247, -0.7343, -1.2745],
        [-1.4805, -0.6955, -1.2958],
        [-1.4523, -0.7124, -1.2891],
        [-1.4187, -0.7404, -1.2692]])
Resultados luego del entrenamiento para la primera frase
tensor([[-1.8107e-02, -4.0842e+00, -6.8054e+00],
        [-7.9985e+00, -2.5249e-03, -6.1258e+00],
        [-6.6842e+00, -5.4931e+00, -5.3800e-03],
        [-8.5010e+00, -9.9927e-03, -4.6315e+00]])


In [ ]:
# Uso del modelo generado

def test_examples(test_data):

   with torch.no_grad():
      inputs = prepare_sequence(test_data, word_to_ix)
      tag_scores = model(inputs)
    
 
   print("FRASE") 
   print("La frase original", test_data)    
   print("La frase original preprocesada", inputs)
   print("Salida del modelo", tag_scores)
   print("Valores máximos e índices", max_values(tag_scores))    
    
print("Clases")
print(tag_to_ix)

#Frase 1
# Las palabras en una oración se pueden etiquetar de tres formas.
# La primera oración tiene 3 palabras "El perro juega"
# por eso el tensor de salida tiene 3 elementos. 
# Cada elemento es un vector de probabilidad de estar asociada a una clase. 
# Es decir hay que calcular la posición del valor máximo. 
#   Ejemplo 1: "El perro juega" ["DET", "NN", "V"]
# Ejemplo: salida 0, 1, 2 con {"DET": 0, "NN": 1, "V": 2} => DET, NN, V 
test_examples(test_data[0][0])


Clases
{'DET': 0, 'NN': 1, 'V': 2}
FRASE
La frase original ['el', 'perro', 'juega']
La frase original preprocesada tensor([ 0,  1, 11])
Salida del modelo tensor([[-1.8107e-02, -4.0842e+00, -6.8054e+00],
        [-7.9985e+00, -2.5249e-03, -6.1258e+00],
        [-6.7163e+00, -3.6417e+00, -2.7801e-02]])
Valores máximos e índices (tensor([-0.0181, -0.0025, -0.0278]), tensor([0, 1, 2]))


In [ ]:
#Frase 2
test_examples(test_data[3][0])

print("valor de las etiquetas", tag_to_ix)

FRASE
La frase original ['el', 'estudiante', 'realiza', 'el', 'examen']
La frase original preprocesada tensor([ 0, 29, 30,  0, 31])
Salida del modelo tensor([[-0.0181, -4.0842, -6.8054],
        [-3.9659, -0.9035, -0.5518],
        [-4.1042, -0.2443, -1.6082],
        [-0.0160, -4.3174, -5.9653],
        [-5.3653, -2.2878, -0.1122]])
Valores máximos e índices (tensor([-0.0181, -0.5518, -0.2443, -0.0160, -0.1122]), tensor([0, 2, 1, 0, 2]))
valor de las etiquetas {'DET': 0, 'NN': 1, 'V': 2}


In [ ]:
# Otra prueba
test_examples(test_data[2][0])
print("valor de las etiquetas", tag_to_ix)

FRASE
La frase original ['maria', 'come', 'pastel']
La frase original preprocesada tensor([13,  2, 28])
Salida del modelo tensor([[-9.3772e+00, -8.6755e-03, -4.7614e+00],
        [-6.5641e+00, -5.1750e+00, -7.0912e-03],
        [-3.0297e-02, -3.9470e+00, -4.5535e+00]])
Valores máximos e índices (tensor([-0.0087, -0.0071, -0.0303]), tensor([1, 2, 0]))
valor de las etiquetas {'DET': 0, 'NN': 1, 'V': 2}


# Referencias 

[1] Guthrie, R. (2017). Tutorial. Sequence Models and Long-Short Term Memory Networks. Recuperado de https://pytorch.org/tutorials/beginner/nlp/sequence_models_tutorial.html

[2] LeCun,Y., Bengio, Y.,  & Hinton, G. (2015). Deep learning. Nature, 521(7553):436.

[3] Brownlee, J. (2017). What Are Word Embeddings for Text?. Recuperado de https://machinelearningmastery.com/what-are-word-embeddings/

[4] Bishop, C (2006). Pattern Recognition and Machine Learning. Springer.